In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# 가장 최대 토큰 수

tokenizer.model_max_length =1024
print(tokenizer.model_max_length)

1024


In [5]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [6]:
tokens

['▁바람',
 '도',
 '▁없는',
 '▁공중에',
 '▁수직',
 '의',
 '▁파',
 '문을',
 '▁내',
 '이며',
 '▁고',
 '요',
 '히',
 '▁떨어지는',
 '▁오동',
 '잎은',
 '▁누',
 '구의',
 '▁발자',
 '취',
 '▁입',
 '니까',
 '.']

In [7]:
tokenizer(input_txt, return_tensors="pt")

{'input_ids': tensor([[10891,  7235,  9712, 49207, 14438,  8143,  9203,  9941,  9094,  9639,
          9065,  8084,  8811, 21215, 34769, 19985,  9669, 10139, 21626,  8408,
          9241, 23775,   389]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index = ['kogpt-2_tokens', "input_IDs"])

In [9]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [10]:
#내친 김에 디코딩 성능도 확인해보깅
max_length = 128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample = False)  #do_sample은 매 단계에서 가장 높은 확률을 가진 토큰을 선택함. True면 temperautre, top_k와 함께 사용 
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


**그리디 서치 디코딩 (Greedy Search Decoding)**  
- 그리디 서치 디코딩은 매 단계에서 가장 높은 확률을 가진 토큰을 선택하는 방법입니다. 이 방법은 매우 간단하고 빠르지만, 항상 최적의 결과를 보장하지는 않습니다.  
- 특징  
    - 결정론적: 매번 동일한 입력에 대해 동일한 출력을 생성합니다.  
    - 빠름: 계산 비용이 적습니다.  
    - 단점: 전체적인 최적의 문장을 생성하지 못할 수 있으며, 지역 최적해(local optimum)에 빠질 가능성이 있습니다.

**빔 서치 디코딩 (Beam Search Decoding)**  
- 빔 서치 디코딩은 여러 개의 경로를 동시에 탐색하여 최적의 문장을 찾는 방법입니다. 이 방법은 그리디 서치보다 더 복잡하지만, 더 나은 결과를 생성할 가능성이 높습니다.  
- 특징  
    - 비결정론적: 빔의 크기(beam width)에 따라 결과가 달라질 수 있습니다.  
    - 복잡함: 여러 경로를 동시에 탐색하므로 계산 비용이 큽니다.  
    - 장점: 더 높은 품질의 텍스트를 생성할 수 있습니다.


In [11]:
# 빔서치
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


In [12]:
# 문장 간의 정합성이나 일관성은 떨어지므로, 샘플링 기법까지 추가
# 빔서치
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=True, temperature = 2.0, top_k = 50)
print(tokenizer.decode(output_beam[0]))


바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 어서 가십시요."
"아니, 무슨 말씀이신지요?"
어이없다는 표정으로 나를 쳐다보며 나는 입을 열지 않았다.
"그게 무슨 뜻인지 말씀해 주십시오. 아까 저는 말씀드린 것처럼 이 세상에서 가장 불행한 존재였습니다."
나는 잠시 망설이다가 고개를 끄덕였다.
"이 세상에 저런 사람이 있을 수가 있습니까. 저렇게 불행한 사람이 한 사람도 없지 않습니까."
"무슨 말씀입니까? 저도 마찬가지입니다. 왜 저를 이렇게


In [13]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=True, temperature = 0.5, top_k = 100)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지 않습니다."
그녀는 고개를 끄덕였다.
"어떻게 된 일입니까?"
아무런 대답도 하지 않은 채 그녀는 한숨을 내쉬었다.
"무슨 말씀을 드려야 할지 모르겠습니다만."
"아니, 그게 무슨 말씀이신지 모르겠군요."
잠시 침묵이 흘렀다.
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
"이제 그만 가봐야겠군."
그녀가 다시 입을 열었다.
"정말 죄송합니다


In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)  #top_p 은 핵샘플링으로, 누적 확률이 p에 도달할 때까지 가장 가능성 높은 토큰들만 고려한다.
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇습니다, 선생님."
"선생님, 저도 선생님을 닮았습니다만, 저는 선생님이 저를 닮았다고 생각합니다만."
그녀는 고개를 끄덕였다.
"아니, 선생님은 저보다 훨씬 똑똑한 분이시지요?"
그녀의 눈에는 눈물이 글썽거렸다.
"그래, 그건 그렇고. 선생님의 얼굴을 봐도 알겠지요."
선생님은 한숨을 내쉬었다.
"어떻게 된 일입니까?


단순한 Causal LM에 불과한 kogpt-2는 생성해 낼 문장의 품질을
디코딩 단계에서 인위적으로 조절해주는 방법 밖에 쓸 수 없습니다.

In [15]:
# RLHF 을 적용해보자!!!
# SFT 시도할 모델에 쓸 데이터셋 살펴보기

import json
data_path_1_SFT = "./seowoo/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl"
with open(data_path_1_SFT, "r", encoding="utf-8-sig") as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

In [16]:
# RM에 사용할 모델에 쓸 데이터셋

data_path_2_RM = "./seowoo/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl"
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [17]:
#PPO 학습 모델 데이터셋 


data_path_3_PPO = "./seowoo/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl"
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

# **1. Supervised Fine-Tuning!!!**

In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [19]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token = '</s>', eos_token = '</s>', unk_token = '</s>', pad_token='</s>', padding_side="right", model_max_length=512,
)

print(tokenizer)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word

In [20]:
# 모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의하기

from typing import Optional, Dict, Sequence

In [21]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100 #무시할 값. seq2seeq 모델을 훈련할 때, 입출력을 구분해야 하기 때문 

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict: #이러면 자료형을 명시할 수 있음
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [22]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:  #입력으로 여러 dict 가 포함된 시퀀스를 받음
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels")) #instances에서 input_ids랑 labels 추출
        input_ids = torch.nn.utils.rnn.pad_sequence(  #패딩 적용해서 시퀀스 길이 맞추기
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100) #손실 계산에서 패딩된 부분 무시하기
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id), #input_ids에서 패딩 토큰이 아닌 위치를 나타내는 마스크 생성
        )

In [23]:
train_dataset = SFT_dataset(data_path_1_SFT="./seowoo/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl", tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [24]:
tokenizer.decode(train_dataset.input_ids[0])


"### Instruction(명령어):\n불고기용 고기 한우에요?\n\n### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>"

In [25]:
# -100을 무시하고 디코딩
decoded_labels = tokenizer.decode([token_id for token_id in train_dataset.labels[0] if token_id != -100])
print(decoded_labels)

'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>


In [26]:
#Training arguments를 사용해 trainer 클래스를 정의하기

training_args = TrainingArguments(
    output_dir="./seowoo/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,  #warmup_steps 값은 전체 학습 단계의 1-10% 사이에 설정합니다.
    prediction_loss_only=True,
    fp16 = True  #16비트 부동 소수점으로 모델 훈련하자. 메모리 사용량 줄이고, 속도 높이기 위해. 
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
#trainer.train()
#model.save_pretrained('./seowoo/KoChatGPT/output_1_SFT')

In [28]:
# 문장 생성능력 확인데스

from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
model_name = './seowoo/KoChatGPT/output_1_SFT'
generator = pipeline('text-generation', model=model_name, tokenizer=tokenizer, device=device)


generation_args = dict(   
    num_beams=4,  #several hypotheses at each time step and eventually chooses the hypothesis that has the overall highest probability for the entire sequence. 
    repetition_penalty=2.0,  #값이 1보다 크면 이미 생성된 토큰의 확률을 낮춤, 1보다 크니까 반복은 크게 줄일 수 있음
    no_repeat_ngram_size=4,  #4개의 연속된 토큰이 이미 생성되면 그 시퀀스는 다시 나타나지 않음 
    eos_token_id=375, # \n   
    max_new_tokens=64,  #생성할 최대 토큰 수
    do_sample=True, #확률 분포에서 샘플링해서 다음 토큰을 선택. 
    top_k=50, #상위 50개의 토큰 중 선택
    early_stopping=True  #빔 서치에서 사용됨. 모든 빔이 끝 토큰에 도달하면 생성을 조기 중단 
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에, 불고기용 고기를 먹을 수 없습니다. 하지만 인터넷 검색을 통해 검색해보시면 맛있게 드실 수 있을 것 같습니다! 감사합니다.\n\n마지막으로, 불고기용 고기는 건강에 좋은 음식 중 하나입니다. 건강한 식습관을

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 41대 부통령직을 수행했습니다.律, please provide more context or information situation.律, Please professed to the translation of this statements of the fir

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 로스앤젤레스에 위치해 있습니다. Coasting of Canada Operator Los Angeles. Corporation Publications, promptant to translation in English Capability. Coast

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 미세먼지 여부를 판단할 수 없습니다. 하지만, 미세먼지 농도가 높을 경우 건강에 좋지 않을 수 있습니다. 따라서 미세먼지 농도를 측정하고 대처 방법을 찾아보는 것이 좋습니다. 또한, 외출 시 마스크를 착용하여 미세먼지를 차단하는 것도 좋은 방법입니다.玄


In [29]:
torch.cuda.empty_cache()

# **2. Reward Model 수정**

In [30]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
# RM에 사용할 모델에 쓸 데이터셋

data_path_2_RM = "./KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl"
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [34]:
import colossalai
print(colossalai.__version__)

0.4.1


In [33]:
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy

/opt/conda/lib/python3.10/site-packages/colossalai/pipeline/schedule/_utils.py:19: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _register_pytree_node(OrderedDict, _odict_flatten, _odict_unflatten)
/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:254: UserWarning: <class 'collections.OrderedDict'> is already registered as pytree node. Overwriting the previous registration.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/colossalai/shardformer/layer/normalization.py:45: UserWarning: Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel
  warnings.warn("Please install apex from source (https://github.com/NVIDIA/apex) to use the fused layernorm kernel")


ModuleNotFoundError: No module named 'colossalai.nn.parallel'

--------

# 여기까지!!입니다...끝!

-------

In [ ]:
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

list_data_dict[:4]

[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [ ]:
# 랜덤으로 completion 2개 골라서 학습시킴
total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:  #예를 들어, [0,1] 같은 경우 
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]: #예를 들어, [0,?,2같은 경우]
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]: #예를 들어, [?, 1,2] 같은 경우
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)


In [ ]:
print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))  # x3이 된 걸 확인할 수 있음. 그도 그럴것이 3C2이기 때문이다. 
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [ ]:
# 훈련셋 만들기
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [ ]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


In [ ]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [ ]:
train_dataset.data[:3]

[{'prompt': '보증금 500 정도인 곳으로 보여 주실 수 있어요?',
  'chosen': '죄송합니다. 저는 AI 어시스턴트이기 때문에, 지역이나 상황에 따라 다양한 보증금 요건을 가진 장소들을 알지 못합니다. 하지만, 인터넷 검색이나 부동산 에이전트와 상담하여 원하는 지역에서 보증금이 500 정도인 장소를 찾을 수 있습니다. 감사합니다.',
  'rejected': '보증금 500 원 이하의 월세 아파트는 매우 드물기 때문에 찾기가 어렵습니다. 가능한 경우는 작은 동네에 있는 작은 아파트를 찾는 것이 좋습니다. 또한, 보증금이 없는 월세 아파트를 찾는 것도 좋은 방법입니다.'},
 {'prompt': '흑고래의 무게는 어느 정도야',
  'chosen': '흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.',
  'rejected': '흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.'},
 {'prompt': '1970년 수정 운동을 통해 시리아의 총리가 된 인물은?',
  'chosen': '바스라 헤즈바르 알-아사르',
  'rejected': '위 한동효와 국내 회사들이 한동효간 국제를 통해 회사들의 인물을 국제로에게 통한 국제를 한동효간 국제로에 국제한 국제를 한동효간 국제로에 국제한 국제를 한동효간 국제로에 국제한 국제'}]

In [ ]:
class PairWiseLoss(nn.Module):
    def forward(self, chosen_reward: torch.Tensor, reject_reward: torch.Tensor) -> torch.Tensor:
        probs = torch.sigmoid(chosen_reward - reject_reward)
        log_probs = torch.log(probs)
        loss = -log_probs.mean()
        return loss

In [ ]:
class RewardModelTrainer:
    def __init__(self, model, strategy, optim, train_dataset, eval_dataset, batch_size, max_epochs):
        self.model = model
        self.strategy = strategy
        self.optim = optim
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.batch_size = batch_size
        self.max_epochs = max_epochs
        self.loss_fn = PairWiseLoss()

    def train_one_epoch(self):
        self.model.train()
        total_loss = 0
        
        for batch in self.train_dataset:
            input_ids, attention_mask, rankings = batch
            input_ids = input_ids.cuda()
            attention_mask = attention_mask.cuda()
            rankings = rankings.cuda()

            self.optim.zero_grad()
            outputs = self.model(input_ids=input_ids.view(-1, input_ids.size(-1)), attention_mask=attention_mask.view(-1, attention_mask.size(-1)))
            rewards = self.model.value_head(outputs.last_hidden_state).squeeze(-1)
            rewards = rewards.view(input_ids.size(0), -1)

            chosen_rewards = rewards.gather(1, rankings[:, 0].long().unsqueeze(-1)).squeeze()
            reject_rewards = rewards.gather(1, rankings[:, -1].long().unsqueeze(-1)).squeeze()

            loss = self.loss_fn(chosen_rewards, reject_rewards)
            loss.backward()
            self.optim.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(self.train_dataset)
        return avg_loss
    
    def evaluate(self):
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in self.eval_dataset:
                input_ids, attention_mask, rankings = batch
                input_ids = input_ids.cuda()
                attention_mask = attention_mask.cuda()
                rankings = rankings.cuda()

                outputs = self.model(input_ids=input_ids.view(-1, input_ids.size(-1)), attention_mask=attention_mask.view(-1, attention_mask.size(-1)))
                rewards = self.model.value_head(outputs.last_hidden_state).squeeze(-1)
                rewards = rewards.view(input_ids.size(0), -1)

                chosen_rewards = rewards.gather(1, rankings[:, 0].long().unsqueeze(-1)).squeeze()
                reject_rewards = rewards.gather(1, rankings[:, -1].long().unsqueeze(-1)).squeeze()

                loss = self.loss_fn(chosen_rewards, reject_rewards)
                total_loss += loss.item()
        avg_loss = total_loss / len(self.eval_dataset)
        return avg_loss
    
    def train(self):
        with self.strategy.model_init_context():
            for epoch in range(self.max_epochs):
                train_loss = self.train_one_epoch()
                eval_loss = self.evaluate()
                print(f'Epoch {epoch + 1}/{self.max_epochs} - Train Loss: {train_loss:.4f} - Eval Loss: {eval_loss:.4f}')
    
    def fit(self, use_lora: int = 0):
        if use_lora:
            print("Using LoRA for training.")
            # LoRA 관련 추가 설정 및 초기화
            # 예를 들어, self.model에 LoRA 레이어 추가
            # LoRA 관련 학습 코드 추가
        else:
            print("Training without LoRA.")
        
        with self.strategy.model_init_context():
            self.train()
    
    

In [ ]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [ ]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

Training without LoRA.
Entering model initialization context
Entering model initialization context
Exiting model initialization context
Exiting model initialization context


KeyError: 'completion_0'

# **2.Reward Model**

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Tokenizer

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from contextlib import contextmanager


class GPTRM_custom(nn.Module):
    def __init__(self, pretrained, tokenizer):
        super().__init__()
        self.gpt = GPT2LMHeadModel.from_pretrained(pretrained)
        self.tokenizer = tokenizer
        self.score = nn.Linear(self.gpt.config.n_embd, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        last_hidden_states = outputs.hidden_states[-1]
        return self.score(last_hidden_states[:, -1, :]).squeeze(-1)

@contextmanager
def model_init_context():
    try:
        torch.cuda.empty_cache()
        yield
    finally:
        torch.cuda.empty_cache()

def train_reward_model(model, train_dataloader, optimizer, device, num_epochs):
    model.train()
    
    accumulate_steps = 4
    for epoch in range(num_epochs):
        total_loss = 0
        optimizer.zero_grad()
        for batch in train_dataloader:
            chosen_ids = batch['chosen_ids'].to(device)
            chosen_mask = batch['chosen_mask'].to(device)
            rejected_ids = batch['rejected_ids'].to(device)
            rejected_mask = batch['rejected_mask'].to(device)

            chosen_reward = model(chosen_ids, chosen_mask)
            rejected_reward = model(rejected_ids, rejected_mask)

            loss = -torch.log(torch.sigmoid(chosen_reward - rejected_reward)).mean()
            
            optimizer.zero_grad()
            loss.backward()
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                
            total_loss += loss.item() * accumulation_steps
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=256,
)


In [ ]:
with open("./seowoo/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl", "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

In [ ]:
 total_data_ranking2chosen

[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'chosen': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'rejected': 'Allow me to answer your question. I know that you are curious about me.'},
 {'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'chosen': '라이언에게 말했다.',
  'rejected': 'Allow me to answer your question. I know that you are curious about me.'},
 {'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'chosen': '라이언에게 말했다.',
  'rejected': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'chosen': '이날 목송에서 구글상위노',
  'rejected': '개포주공아파트는 다섯 단지로 이루어져 있습니다.'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'chosen': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'rejected': '개포주공아파트는 다섯 단지로 이루어져 있습니다.'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'chosen': '이날 목송에서 구글상위노',
  'rejected': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'chosen': 'The diameter of

In [ ]:
print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [ ]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [ ]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

1000
200


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class RewardDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = item['prompt']
        chosen = item['chosen']
        rejected = item['rejected']

        # 토큰화 및 인코딩
        chosen_encodings = self.tokenizer(prompt + chosen, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        rejected_encodings = self.tokenizer(prompt + rejected, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'chosen_ids': chosen_encodings['input_ids'].squeeze(),
            'chosen_mask': chosen_encodings['attention_mask'].squeeze(),
            'rejected_ids': rejected_encodings['input_ids'].squeeze(),
            'rejected_mask': rejected_encodings['attention_mask'].squeeze()
        }

In [ ]:
# RewardDataset 인스턴스 생성
reward_dataset = RewardDataset(total_data_ranking2chosen, tokenizer)

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


In [ ]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [ ]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

class RewardModelTrainer:
    def __init__(self, model, optimizer, train_dataset, eval_dataset, batch_size, max_epochs, strategy):
        self.strategy = strategy
        self.model = self.strategy.prepare_model(model)
        self.optimizer = optimizer
        self.train_dataloader = self.strategy.prepare_dataloader(DataLoader(train_dataset, batch_size=batch_size, shuffle=True))
        self.eval_dataloader = self.strategy.prepare_dataloader(DataLoader(eval_dataset, batch_size=batch_size, shuffle=False))
        self.max_epochs = max_epochs
        self.device = self.strategy.get_device()

    def train(self):
        self.model.train()
        for epoch in range(self.max_epochs):
            total_loss = 0
            for batch in self.train_dataloader:
                chosen_ids = batch['chosen_ids'].to(self.device)
                chosen_mask = batch['chosen_mask'].to(self.device)
                rejected_ids = batch['rejected_ids'].to(self.device)
                rejected_mask = batch['rejected_mask'].to(self.device)

                chosen_reward = self.model(chosen_ids, chosen_mask)
                rejected_reward = self.model(rejected_ids, rejected_mask)

                loss = -torch.log(torch.sigmoid(chosen_reward - rejected_reward)).mean()
                
                self.optimizer.zero_grad()
                self.strategy.backward(loss)
                self.strategy.optimizer_step(self.optimizer)

                total_loss += loss.item()

            avg_loss = total_loss / len(self.train_dataloader)
            print(f"Epoch {epoch+1}/{self.max_epochs}, Training Loss: {avg_loss:.4f}")

    def evaluate(self):
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in self.eval_dataloader:
                chosen_ids = batch['chosen_ids'].to(self.device)
                chosen_mask = batch['chosen_mask'].to(self.device)
                rejected_ids = batch['rejected_ids'].to(self.device)
                rejected_mask = batch['rejected_mask'].to(self.device)

                chosen_reward = self.model(chosen_ids, chosen_mask)
                rejected_reward = self.model(rejected_ids, rejected_mask)

                loss = -torch.log(torch.sigmoid(chosen_reward - rejected_reward)).mean()
                total_loss += loss.item()

        avg_loss = total_loss / len(self.eval_dataloader)
        print(f"Evaluation Loss: {avg_loss:.4f}")

In [ ]:
import torch

class NaiveStrategy:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def model_init_context(self):
        class ModelInitContext:
            def __enter__(self):
                torch.cuda.empty_cache()

            def __exit__(self, exc_type, exc_val, exc_tb):
                torch.cuda.empty_cache()

        return ModelInitContext()

    def prepare_model(self, model):
        return model.to(self.device)

    def prepare_dataloader(self, dataloader):
        return dataloader

    def backward(self, loss):
        loss.backward()

    def optimizer_step(self, optimizer):
        optimizer.step()

    def get_device(self):
        return self.devicea

In [ ]:
# 트레이너 초기화
trainer = RewardModelTrainer(
    model=model,
    optimizer=optimizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    batch_size=4,
    max_epochs=1,
    strategy=strategy
)

NameError: name 'optimizer' is not defined